# Chapter 6 Appendix

## Linking Models

It's a problem when we need to combine models from different investment universe, could be different asset class (equity vs. bond), countries, etc. There are three ways:

### Integrated modeling
model assets across universes as if they are in the same universe, straightforward.
  - Biggest problem is asynchronous in returns: different universe may have different trading and holiday schedules, thus missing returns when we aggregate returns across universes, solutions:
    - Use lower frequency (e.g. weekly data instead of daily data), less observations and higher volatility (order of sqrt of n)
    - Asynchronicity induces cross-autocorrelations, so we can add the autocorrelations back into the covariance matrix estimate using the Newey-West estimator
    - We can estimate a global covariance matrix, then fill in the pairwise covariances using more complete factor returns, finally shrink off-diagonally to make it positive-deinite, use at your own risk!

### Separate modeling
Model each universe first, then combine them for a global mode. 

First model local covariance matrices for each universe $\hat{\Omega}_f^{(1)},\ldots,\hat{\Omega}_f^{(q)}$ using the full factor returns in each local universe. 

Then estimate the global covariance matrix $\hat{\Omega}_f^{(I)}$ using the **joint** factor returns across all universes, which could be partitioned in to blocks corresponding to single local covariance matrices $V_{i,i}$ and cross-universe covariance matrices $V_{i,j}$. Note that $V_{i,i} \neq \hat{\Omega}_f^{(i)}$, because one uses all factor returns while the other uses a subset due to missing returns from other universes (think inner-join across factor returns with missing values). However, they should be similar to each other. The global idio covariance matrix is block-diagonal with $diag(\hat{\Omega}_{\epsilon}^{(1)},\ldots,\hat{\Omega}_{\epsilon}^{(q)})$, the assumption is that idio returns are uncorrelated across universes. 

Finally, the factor loadings of the global model is also block-diagonal since the asset returns in one universe shouldn't depend on factors from another universe. 

To deal with the issue that $V_{i,i} \neq \hat{\Omega}_f^{(i)}$, we use a corrective term $C^{(i)} = V_{i,i}^{-1/2}(\hat{\Omega}_f^{(i)})^{1/2}$. The global corrective matrix is $C = diag(C^{(i)},\ldots,C^{(q)}$, then $\hat{\Omega}_f^{(IR)}=C^TVC$. The corrective term essentially recovers the local covariance sub-matrices in the global covariance matrix, since $V$ should be close to $\hat{\Omega}$, it's not a full-on factor rotation and we don't need to apply this to factor loadings. Finally, we shrink the off-diagnoal covariance sub-matrices with a parameter $\rho$.

### Multistage modeling
Model common factors across universes, then model the residuals in each universe. The key is to find all global factors such that the local factor returns are uncorrelated conditioned on the global factors. 

First gather global characteristics $B^{(G)}$ (think country factor, market factor, style factors like momentum, volatility and value), regress intergrated universe assert returns on $B^{(G)}$ and get the global factor returns $g_t$.

Then for each local universe, orthogonalize the factor returns on the global factor returns (use WLS if suspect heteroskedasticity):
$$f_t^{(i)} = C^{(i)}g_t + \tilde{f}_t^{(i)}$$

Thirdly, the integrated factor model can be estimated. It's very similar to seperate modeling, we just seperate the factor returns into global factor returns and local factor returns. The covariance matrix in case is then a combination of global covariance matrix and a block-diagonal orthogonalized local factor covariance matrix. Although we must verify that the orthogonalized local factor returns are uncorrelated across universe, otherwise the local factor covariance matrix can't be block-diagonal. 

The tricky part is to find the set of global characteristics that include all common information across the universes such that the tilde local factor returns are uncorrelated across universe.


## Currency Rebasing

$$(1 +r^f_{EUR})(1+g_{EURUSD}) - 1 - r^f_{USD} \simeq g_{EURUSD} + r^f_{EUR} - r^f_{USD} = r^{cf}_{EURUSD}$$

This is the currency return adjusted by the difference in risk-free rate, gain in currency value plus the interest rate difference. Plug in the local returns:

$$(r^l_{EUR} - r^f_{EUR}) + (g_{EURUSD} - r^f_{USD} + r^f_{EUR}) = r^e_{EUR} + r^{cf}_{EURUSD}$$

So the investment in foreign currency is the excess return over the risk-free local interest rate, plus the currency return adjusted by risk-free rate difference.

To generalize, say we have three currencies j,k and l, then:

$$r^{cf}_{j,k} + r^{cf}_{k,l} = g_{j,k} + r^f_{j} - r^f_{k} + g_{k,l} + r^f_{k} - r^f_{l} = g_{j,k} + g_{k,l} + r^f_{j} - r^f_{l} = r^{cf}_{j,l}$$

To change base from currency k to currency l, which is just a generalization of the above formula:

$$r^{cf}_{\dot, k} = (I_q - A^{(k)})r^{cf}_{\dot,l}$$

$$ [A^{(k)}]_{m,n} = \begin{cases}
0, n \neq k \\
1, n = k
\end{cases}$$

Put everything together:

$$r = r^e + Cr^{cf}_{\dot,k}=Bf+\epsilon + C(I_q - A^{(k)})r^{cf}_{\dot,l}$$

Key assumptions:
- Factor model ignored the currency returns
- Assumes zero covariance between factors and currency returns

## Factors

- Markets: Most pervasive, beta to market (SPX, RUA, etc.)
- Contries and Industries: binary encoder
- Momentum: stocks trend
- Reversal: stocks reverse in shorter term
- Fundamental valuation: financial reports derivations
- Low Beta/Low Vol: high beta stocks have lower risk-adjusted performance than low-beta stocks
- Liquidity: iliquid stocks outperform
- Crowding: forced liquidations have cascading effect
- Size: small firms outperform